TF-IDF Classifier

Я навчав класифікатор який буде знаходити "токсичні" коментарі

Дані можна завантажити тут - https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/data


In [1]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [2]:
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

train = pd.read_csv('train.csv').fillna(' ')
test = pd.read_csv('test.csv').fillna(' ')
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


Стадартнимі підходами для аналізу тексту є [Bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) і його модифікація [TF-IDF](https://en.wikipedia.org/wiki/Tf%E2%80%93idf).

Вони реалізовані в `sklearn` у вигляді [CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) и [TfidfVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html).
Датасет складається з коментарів та розмічених данних по стовпцях, до якого класу коментар відноситься:'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate' - що можна наглядно побачити вище ( output train.head()).

In [3]:
train_text = train['comment_text']
test_text = test['comment_text']
all_text = pd.concat([train_text, test_text])

In [ ]:
Використовуємо countVectorizer для того, щоб порахувати всі слова з датасету і скласти словник з них.

In [4]:
cnt_vect = CountVectorizer()
cnt_vect.fit(all_text)
matrix = cnt_vect.transform(all_text)
idx = np.argsort(np.sum(matrix, axis=0))[-1]
inv_vocb = {v: k for k, v in cnt_vect.vocabulary_.items()}
idx

matrix([[176983, 206793, 206795, ..., 206804, 287217, 283352]])

In [5]:
inv_vocb[283352]

'the'

З тексту робимо векторну репрезентацію слів. 
За допомогою алгоритмів Tfidf створюємо векторну реперезентацію для слів і для н-грам

In [6]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern= r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 1),
    max_features=50000)
word_vectorizer.fit(all_text)
train_word_features = word_vectorizer.transform(train_text)
test_word_features = word_vectorizer.transform(test_text)
train_word_features

<159571x50000 sparse matrix of type '<class 'numpy.float64'>'
	with 4032516 stored elements in Compressed Sparse Row format>

In [8]:
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    stop_words='english',
    ngram_range=(4, 6),
    max_features=50000)
char_vectorizer.fit(all_text)
train_char_features = char_vectorizer.transform(train_text)
test_char_features = char_vectorizer.transform(test_text)


/usr/local/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:520: UserWarning: The parameter 'stop_words' will not be used since 'analyzer' != 'word'
  warnings.warn("The parameter 'stop_words' will not be used"


In [9]:
import scipy as s
train_features = s.sparse.hstack([train_char_features, train_word_features])
test_features = s.sparse.hstack([test_char_features, test_word_features])

Для класифікації коментаря я використовую логістичну регресію [LogisticRegression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html).

Будемо тренувати по одному класифікатору на кожен клас.
Що б провалідувати якість моделі скористаємося функцією [cross_val_score](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html)

Також під час тренування спробуємо знайти найкращий регуляризаційний параметр С, пробуємо один 5 параметрів і дивимося, який дасть найкращий score


In [11]:
scores= []
C = [0.1, 0.5, 1.0, 1.5, 2.0]
for c in C:
    classifier = LogisticRegression(penalty='l2',C=c, solver = 'newton-cg', class_weight= 'balanced', n_jobs = -1)
    for class_name in class_names:
        train_target = train[class_name]
        
        cv_score = np.mean(cross_val_score(classifier, train_features, train_target, cv=3, scoring='roc_auc'))
        #print('CV score for class {} is {}'.format(class_name, cv_score))
        scores.append(cv_score)

    print('Total score is {} for c= {}'.format(np.mean(scores), c))

Total score is 0.9833108834281727 for c= 0.1
Total score is 0.9842437424945167 for c= 0.5
Total score is 0.9844937013133167 for c= 1.0
Total score is 0.9845231238789421 for c= 1.5
Total score is 0.9844602154662377 for c= 2.0


Бачимо, що при будь-якому параметрі С класифікатори видають майже однаковий score

In [12]:
submission = pd.DataFrame.from_dict({'id': test['id']})

In [13]:
classifier = LogisticRegression(penalty='l2',C= 0.1, solver = 'newton-cg',
                                class_weight= 'balanced',max_iter=1000,
                                tol=0.00001, n_jobs = -1)
for class_name in class_names:
    train_target = train[class_name]
    classifier.fit(train_features,train_target)
    submission[class_name] = classifier.predict_proba(test_features)[:, 1]    

In [14]:
submission.to_csv('submission.csv', index=False)
print("Have Done!!!")

Have Done!!!
